how could i express model level ie overall cold start vulnerability. say for someone selecting between 2 models and they want to know which will be less vulnerable to cold start. 

define this only based off of davis and then show that see this model has a lower csv(as defined by my metric ) and does better on pharos than this other model that i also do




In [ ]:
import numpy as np
import pandas as pd
from shapely.geometry import Point, Polygon
from scipy.spatial import ConvexHull

def compute_CHR_model_level_global(merged_df, min_points=3):
    """
    Computes model-level CHR as a single value:
    Fraction of all blinded points that fall outside
    the convex hull of all unblinded points.
    """
    points_unblinded = merged_df[["True_Label", "Unblinded_Prediction"]].values
    points_blinded = merged_df[["True_Label", "Blinded_Prediction"]].values

    if len(points_unblinded) < min_points:
        raise ValueError("Not enough unblinded points to construct convex hull.")

    try:
        hull = ConvexHull(points_unblinded)
        polygon = Polygon(points_unblinded[hull.vertices])

        inside_mask = np.array([polygon.contains(Point(p)) for p in points_blinded])
        chr_value = (~inside_mask).sum() / len(points_blinded)
        return chr_value

    except Exception as e:
        print(f"Convex hull computation failed: {e}")
        return np.nan



def compute_CSPD_model_level_global(merged_df, min_affinity=5):
    """
    Computes model-level CSPD as the mean absolute normalized prediction deviation (NPD)
    across all points with True_Label ≥ min_affinity.
    """
    df = merged_df.copy()
    df = df[df["True_Label"] >= min_affinity]

    # Avoid division by zero
    df = df[df["True_Label"] != 0]

    if df.empty:
        raise ValueError("No valid data points after applying min_affinity filter.")

    df["NPD"] = (df["Unblinded_Prediction"] - df["Blinded_Prediction"]) / df["True_Label"]
    cspd_value = df["NPD"].abs().mean()
    return cspd_value



def compute_CSVS_model_level_global(merged_df, min_affinity=5, alpha=None, beta=None):
    """
    Computes model-level CSVS using global (non-grouped) CSPD and CHR.
    Returns:
        - scalar CSVS value
        - CSPD scalar
        - CHR scalar
        - alpha, beta used
    """
    # Get global CSPD and CHR
    CSPD = compute_CSPD_model_level_global(merged_df, min_affinity)
    CHR = compute_CHR_model_level_global(merged_df, min_affinity)

    # Normalize weights
    if alpha is None:
        alpha = 1 / CSPD if CSPD != 0 else 0
    if beta is None:
        beta = 1 / CHR if CHR != 0 else 0

    # Compute scalar CSVS
    csvs_model = alpha * CSPD + beta * CHR

    return csvs_model, CSPD, CHR, alpha, beta

